In [41]:
import tarski
import tarski.evaluators
from tarski.theories import Theory
from tarski.io.fstrips import ParsingError, FstripsReader
from tarski.syntax import *

# Parsing and Processing PDDL

In [42]:
reader = FstripsReader(raise_on_error=True)

In [43]:
reader.parse_domain('probBLOCKS-domain.pddl')
reader.parse_instance('probBLOCKS-4-2.pddl')

In [44]:
L = reader.problem.language

We can retrieve any sort (or _type_ in PDDL-parlance), constant, function or predicate 
via the `get` method of the class `Language`. Since we are dealing
with the classic `Blocks World` benchmark we know there must be predicates `on(x,y)` and `clear(x)` 

In [45]:
on = L.get('on')
clear = L.get('clear')

`Language` instances allow us to iterate over the list of declared predicates etc. via the
properties `predicates`, `functions`

In [46]:
for pred in L.predicates:
    print(str(pred))

=/2
!=/2
on/2
ontable/1
clear/1
handempty/0
holding/1


As seen above `Tarski` parser defines the equality predicate and its negation, as  well as the
usual `on` etc. The number after the name of the predicate indicates its _arity_ (i.e. number of arguments).

Getting our hands on the objects defined in the instance file `probBLOCKS-4-2.pddl` requires to 
navigate the type hierarchy. First, we can take a look at what are the sorts defined similarly as
we do for predicates

In [47]:
for S in L.sorts:
    print(str(S))

Sort(object)


There is only one sort defined, `object`. We can check out its elements 

In [48]:
objects = L.get('object')
for obj in objects.domain():
    print(str(obj), type(obj))

c <class 'tarski.syntax.terms.Constant'>
a <class 'tarski.syntax.terms.Constant'>
d <class 'tarski.syntax.terms.Constant'>
b <class 'tarski.syntax.terms.Constant'>


### Excursion: Indexing Logical Elements

It is handy to have the objects indexed in a table for easy retrieval. We can have such a table
by name

In [49]:
obj_table1 = { str(obj): obj for obj in objects.domain()}
obj_table1

{'c': c (object), 'a': a (object), 'd': d (object), 'b': b (object)}

Alternatively, we can use the constants themselves as keys in a dictionary, by wrapping them with
an adapter in this way

In [50]:
obj_table2 = { symref(obj): [] for obj in objects.domain()}
obj_table2

{symref[c]: [], symref[a]: [], symref[d]: [], symref[b]: []}

Wrapping constant, terms and atoms is necessary as `Tarski` overloads most of the standard Python 
operators to provide compact syntax for formulas and expressions. For instance, if we wanted
to compare whether two constants are the same _value_ one could try the following

In [51]:
x = L.get('a')
y = L.get('b')
x == y

=(a,b)

Yet using the comparison operator `==` results in the _atom_ `=(a,b)`, as `a` and `b` stand for
two elements of a first-order language. If we use the wrappers

In [52]:
x = L.get('a')
y = L.get('b')
symref(x) == symref(y)

False

Then we get to compare the actual _physical_ objects than the _logical elements_ (e.g.
 constants, arithmetic expressions, Boolean formulas) they represent.

## Grounding    

`Tarski` supports two different strategies for grounding lifted representations of STRIPS and ADL. One is based around
the idea by Malte Helmert of constructing a logic program and computing its models so as to determine the reachability
of a ground atom _online_. This is strategy constrasts with the classical approach, which we refer to as `naive`, where 
all potential groundings are first enumerated and then tested by reachability. While Helmert's approach is slower
for smaller problems, it does pay off greatly for instances with thousands of ground actions, or more, which can only
rarely be processed even by optimized implementations of the `naive` approach.

To implement Helmert's algorithm, or `lp` from now on, we rely on the excellent tools developed by for Answer Set
Programming, and inparticular, `gringo`, the suite of grounding tools for
disjunctivel logic programs. Stable versions of tools are neatly packaged in Ubuntu 18.04 so you can install them
like this

```
$ sudo apt install gringo
```

We can get hold of the classes encapsulating the grounding process from the `tarski.grounding` module

In [53]:
from tarski.grounding import LPGroundingStrategy
from tarski.grounding.errors import ReachabilityLPUnsolvable

and ground the instance of Blocks World with a one-liner

In [54]:
try:
    grounding = LPGroundingStrategy(reader.problem)
except ReachabilityLPUnsolvable:
    print("Problem was determined to be unsolvable during grounding")

Note that the `lp` grounder can determine if an instance is unsolvable, so we need to handle that possible exception.

We can inspect the results of the grounding process with ease. For the ground actions, 
we can query the `grounding` object for a dictionary that maps every action schema to
a list tuples of object names that schema variables are to be bound to, as they were 
found to be reachable

In [55]:
actions = grounding.ground_actions()

for name, bindings in actions.items():
    print('Action schema', name, 'got', len(bindings), 'bindings')
    for b in bindings:
        print(b)

Action schema pick-up got 4 bindings
('b',)
('a',)
('c',)
('d',)
Action schema put-down got 4 bindings
('b',)
('a',)
('c',)
('d',)
Action schema stack got 16 bindings
('a', 'c')
('a', 'b')
('b', 'b')
('d', 'a')
('c', 'a')
('b', 'd')
('d', 'd')
('d', 'c')
('c', 'd')
('c', 'c')
('b', 'c')
('c', 'b')
('d', 'b')
('a', 'a')
('b', 'a')
('a', 'd')
Action schema unstack got 16 bindings
('a', 'c')
('a', 'b')
('b', 'b')
('d', 'a')
('c', 'a')
('b', 'd')
('d', 'd')
('d', 'c')
('c', 'd')
('c', 'c')
('b', 'c')
('c', 'b')
('d', 'b')
('a', 'a')
('b', 'a')
('a', 'd')


The astute reader may be now wondering why we only return the _bindings_ rather
than the grounded actions themselves. The reason is that doing so is not safe
in general, as big instances (such as those commonly found on the IPC-18 benchmarks)
result in thousands of ground operators, and is possible to exhaust the memory
available to the Python interpreter. 

To ameliorate that issue, we settle for returning instead the minimal amount of data
necessary so users can decide how to instantiate ground operators efficiently.  

To access the ground atoms, or _state variables_, identified during the grounding
process, we use a similar interface

In [56]:
lpvariables = grounding.ground_state_variables()
for atom_index, atoms in lpvariables.enumerate():
    print('p_{}:'.format(atom_index), atoms)

p_0: clear(b)
p_1: clear(a)
p_2: clear(c)
p_3: clear(d)
p_4: handempty()
p_5: holding(b)
p_6: holding(a)
p_7: holding(d)
p_8: holding(c)
p_9: ontable(b)
p_10: ontable(a)
p_11: ontable(c)
p_12: ontable(d)
p_13: on(a,d)
p_14: on(a,b)
p_15: on(b,b)
p_16: on(c,a)
p_17: on(d,a)
p_18: on(b,c)
p_19: on(d,c)
p_20: on(c,c)
p_21: on(c,d)
p_22: on(d,d)
p_23: on(b,d)
p_24: on(c,b)
p_25: on(d,b)
p_26: on(a,a)
p_27: on(b,a)
p_28: on(a,c)


We note that in this case we _do_ return the actual grounded language element. This
is because the number of fluents is not generally subject to the same kind of 
combinatorial explosion that ground actions are. **This assessment may change
in the future**.

## The $K_0$ compilation

For the purpose of this tutorial we will look at the seminal paper by Palacios and Geffner
"Compiling Uncertainty Away in Conformant Planning Problems with Bounded Width", and see how
we can implement the $K_0$ compilation of conformant into classical problems.

*Definition* (Translation $K_0$). For a conformant planning problem $P=\langle F,I,O,G\rangle$, the 
translation $K_0(P) = \langle F', I', O', G'\rangle$ is classical planning problem with
 - $F' = \{ KL, K\neg L\, \mid\, L \in F \}$
 - $I' = \{ KL \, \mid \, L \text{ is a unit clause in } I\}$
 - $G' = \{ KL \, \mid \, L \in G\}$
 - $O' = O$ but with each precondition $L$ for $a \in O$ replaced by $KL$, and each conditional effect 
 $a: C \rightarrow L$ replaced by $a: KC \rightarrow KL$ and $a: \neg K \neg C \rightarrow \neg K \neg L$.

### Constructing the fluent set $F'$

To construct the set of fluents $F'$ we need to create a fresh first-order language
to accomodate the new symbols

In [57]:
KP_lang = tarski.language("K0(P)", theories=[Theory.EQUALITY])

In this tutorial we will explore a compact encoding enabled by the modeling capabilities
of `tarski`. We start creating a sort (type) for the literals of each of the atoms
in the original conformant problem $P$. We call this type `P-literals`

In [58]:
p_lits = KP_lang.sort("P-literals", KP_lang.Object)

 
Now we enumerate the atoms in $F$ and we keep matching lists `P_lits` and `reified_P_lits`
that will facilitate later the compilation process

In [59]:
P_lits = []
reified_P_lits = []
for atom_index, atoms in lpvariables.enumerate():
    f_atom = Atom(atoms.symbol, atoms.binding)
    fp_lit = f_atom
    fn_lit = neg(f_atom)
    P_lits += [fp_lit, fn_lit]
    reified_P_lits += [KP_lang.constant(str(fp_lit), p_lits), KP_lang.constant(str(fn_lit), p_lits)]

so we end up, for every pair of literals $L$, $\neg L$, with _objects_ "$L$" and
"$\neg l$". To obtain $F'$ we need to add a new predicate

In [60]:
K = KP_lang.predicate("K", p_lits)

from which we can easily define the $KL$, $K\negL$, $\neg K L$ 
and $\neg K \neg L$ literals

In [61]:
K(reified_P_lits[0])

K(clear(b))

In [62]:
neg(K(reified_P_lits[0]))

(not K(clear(b)))

In [63]:
K(reified_P_lits[1])

K((not clear(b)))

In [64]:
neg(K(reified_P_lits[1]))

(not K((not clear(b))))

### Excursion: Inspecting grounded initial states

We go on a little excursion to show how we can enumerate the literals that
are true in the initial state of a grounded STRIPS problem. First, we need
to select what algorithm we want to use to evaluate expressions

In [65]:
reader.problem.init.evaluator = tarski.evaluators.simple.evaluate

The `simple` evaluator is a straightforward depth-first traversal that processes 
 each of the nodes of the tree formed by the syntactic elements of the expression 
 to be evaluated, returning either a expression or a value.

Once the evaluator algorithm is selected, we can use the random access iterator
to evaluate expressions as we do below

In [66]:
I = []
for atom_index, atoms in lpvariables.enumerate():
    atom = Atom(atoms.symbol, atoms.binding)
    if reader.problem.init[atom]:
        I += [atom]
    else:
        I += [neg(atom)]

 to obtain the list of literals true in the initial state.

In [67]:
for p in I:
    print(p)


(not clear(b))
clear(a)
clear(c)
clear(d)
handempty()
(not holding(b))
(not holding(a))
(not holding(d))
(not holding(c))
ontable(b)
ontable(a)
(not ontable(c))
ontable(d)
(not on(a,d))
(not on(a,b))
(not on(b,b))
(not on(c,a))
(not on(d,a))
(not on(b,c))
(not on(d,c))
(not on(c,c))
(not on(c,d))
(not on(d,d))
(not on(b,d))
on(c,b)
(not on(d,b))
(not on(a,a))
(not on(b,a))
(not on(a,c))


### Constructing the initial state $I'$

For the purpose of this tutorial, we will consider a quite difficult conformant
problem, where the only information we have initially is that the robot hand is empty.

In order to construct that initial state, we need to access the relevant predicate and
object symbols 

In [68]:
handempty = reader.problem.language.get('handempty')
holding = reader.problem.language.get('holding')
a, b, c, d = reader.problem.language.get('a', 'b', 'c', 'd')

so we can write directly the literals corresponding to the specification above.

In [29]:
I = [handempty(), neg(holding(a)), neg(holding(b)), neg(holding(c)), neg(holding(d))]

We obtain $I'$ by first computing the set of literals of the original conformant
problem $P$ that are unit clauses

In [30]:
unit_clauses = set()
K_I = []
for l0 in I:
    p_l0 = KP_lang.get(str(l0))
    K_I += [K(p_l0)]
    unit_clauses.add(symref(l0))

We use the set `unit_clauses` to then determine which $\neg K L$ and $\neg K \neg L$
fluents we need to have in our initial state as well

In [31]:
for atom_index, atoms in lpvariables.enumerate():
    lp = Atom(atoms.symbol, atoms.binding)
    p_lp = KP_lang.get(str(lp))
    ln = neg(lp)
    p_ln = KP_lang.get(str(ln))
    if lp not in unit_clauses:
        K_I += [neg(K(p_lp))]
    if ln not in unit_clauses:
        K_I += [neg(K(p_ln))]

In [32]:
for k_p in K_I:
    print(k_p)

K(handempty())
K((not holding(a)))
K((not holding(b)))
K((not holding(c)))
K((not holding(d)))
(not K(clear(b)))
(not K((not clear(b))))
(not K(clear(a)))
(not K((not clear(a))))
(not K(clear(c)))
(not K((not clear(c))))
(not K(clear(d)))
(not K((not clear(d))))
(not K(handempty()))
(not K((not handempty())))
(not K(holding(b)))
(not K((not holding(b))))
(not K(holding(a)))
(not K((not holding(a))))
(not K(holding(d)))
(not K((not holding(d))))
(not K(holding(c)))
(not K((not holding(c))))
(not K(ontable(b)))
(not K((not ontable(b))))
(not K(ontable(a)))
(not K((not ontable(a))))
(not K(ontable(c)))
(not K((not ontable(c))))
(not K(ontable(d)))
(not K((not ontable(d))))
(not K(on(a,d)))
(not K((not on(a,d))))
(not K(on(a,b)))
(not K((not on(a,b))))
(not K(on(b,b)))
(not K((not on(b,b))))
(not K(on(c,a)))
(not K((not on(c,a))))
(not K(on(d,a)))
(not K((not on(d,a))))
(not K(on(b,c)))
(not K((not on(b,c))))
(not K(on(d,c)))
(not K((not on(d,c))))
(not K(on(c,c)))
(not K((not on(c,c))))
(

### Constructing the goal state $G'$

Constructing the goal state proceeds very much as for initial states, but 
way simpler, as we do not need to complete with the logical implications of
literals as we do for initial states

In [33]:
on, ontable = reader.problem.language.get('on', 'ontable')
G = [clear(a), on(a, b), on(b, c), on(c, d), ontable(d)]

In [34]:
K_G = []
for l_G in G:
    p_l_G = KP_lang.get(str(l_G))
    K_G += [K(p_l_G)]

### Constructing the action set $O'$

Constructing the set of operators is a bit more involved. We start importing
a helper function, `ground_schema`, that instantiates action schemas as per the
given variable binding.

In [35]:
from tarski.syntax.transform.action_grounding import ground_schema

We use the same interface discussed above, to get access to the set of bindings
identified by the grounding procedure, and just call the helper function on the
schemata and the bindings.

In [36]:
O = []
for name, ops in actions.items():
    print('Action schema', name, 'got', len(ops), 'ground actions')
    schema = reader.problem.get_action(name)
    print(list(schema.parameters.vars()))
    for op in ops:
        ground_action = ground_schema(schema, op)
        O += [ground_action]
        print(ground_action)
        print(ground_action.precondition, ground_action.effects)

Action schema pick-up got 4 ground actions
[?x (object)]
pick-up(b)()
(clear(b) and ontable(b) and handempty()) [(T -> DEL(ontable(b))), (T -> DEL(clear(b))), (T -> DEL(handempty())), (T -> ADD(holding(b)))]
pick-up(a)()
(clear(a) and ontable(a) and handempty()) [(T -> DEL(ontable(a))), (T -> DEL(clear(a))), (T -> DEL(handempty())), (T -> ADD(holding(a)))]
pick-up(c)()
(clear(c) and ontable(c) and handempty()) [(T -> DEL(ontable(c))), (T -> DEL(clear(c))), (T -> DEL(handempty())), (T -> ADD(holding(c)))]
pick-up(d)()
(clear(d) and ontable(d) and handempty()) [(T -> DEL(ontable(d))), (T -> DEL(clear(d))), (T -> DEL(handempty())), (T -> ADD(holding(d)))]
Action schema put-down got 4 ground actions
[?x (object)]
put-down(b)()
holding(b) [(T -> DEL(holding(b))), (T -> ADD(clear(b))), (T -> ADD(handempty())), (T -> ADD(ontable(b)))]
put-down(a)()
holding(a) [(T -> DEL(holding(a))), (T -> ADD(clear(a))), (T -> ADD(handempty())), (T -> ADD(ontable(a)))]
put-down(c)()
holding(c) [(T -> DEL(hol

Creating the precondition and effect formulas of the operators in $O'$ requires
1) creating copies of a ground operator, 2) substitute formulas (i.e. wherever
it says $L$ it needs to say $KL$) and 3) create new add and del effects for the
new operators. 

In [37]:
import copy
from tarski.syntax.transform.substitutions import substitute_subformula
from tarski.fstrips import Action, AddEffect, DelEffect

We start creating a dictionary where we map literals $L$ to their corresponding
$K$-literal

In [38]:
subst = {}
for p, Kp in zip(P_lits, [K(p) for p in reified_P_lits]):
    subst[symref(p)] = Kp

note the role played by the two lists `P_lits` and `reified_P_lits` we created
above.

We note that any effect, the construction of the $KC$ and $KL$ **formulas** 
is always the same, so we introduce a function that does the translation

In [39]:
def make_K_condition_and_effect(eff, K_prec):
    KC = [K_prec]
    if not isinstance(eff.condition, Tautology):
        KC += [substitute_subformula(copy.deepcopy(eff.condition), subst)]
    KC = land(*KC)
    KL = subst[symref(eff.atom)]
    
    return KC, KL

We finally put together the substitution rule for the $P$-literals, and construct
the new set of operators as follows

In [40]:
K_O = []
for op in O:
    K_prec = substitute_subformula(copy.deepcopy(op.precondition), subst)
    K_effs = []
    for eff in op.effects:
        KC, KL = make_K_condition_and_effect(eff, K_prec)
        if isinstance(eff, AddEffect):
            K_effs += [AddEffect(KC, KL)]
        elif isinstance(eff, DelEffect):
            K_effs += [DelEffect(KC, KL)]
        else:
            raise RuntimeError("Effect type not supported by compilation!")
    K_O += [Action(KP_lang, op.name, VariableBinding(), K_prec, K_effs)]
        


TypeError: __init__() missing 1 required positional argument: 'effects'